<a href="https://colab.research.google.com/github/mattrothery/stylegan3-notebook/blob/main/Stylegan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stylegan3 Notebook

Finetune premade models on a custom image dataset!

In [ ]:
# import torch
# print(torch.__version__)
# !pip uninstall torch -y
# !pip install torch==1.9.0

import torch
print(torch.__version__)

1.10.0+cu111


In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git

Cloning into 'stylegan3'...
remote: Enumerating objects: 193, done.
remote: Total 193 (delta 0), reused 0 (delta 0), pack-reused 193
Receiving objects: 100% (193/193), 4.18 MiB | 17.74 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
!nvidia-smi

Thu Jan 27 15:27:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Training data tools

Use the following code cells to unzip training images from Google Drive to a folder, remove any non-image files, then resize all to a set resolution. Must first mount drive in the files sidebar to the left. The training directory can then be passed to the dataset tool to put them in the right format for training.

In [ ]:
# # Unzip training images to a directory
# !mkdir train
# !unzip drive/MyDrive/train.zip -d train

In [ ]:
# import os, sys

# path = "train/"
# dirs = os.listdir(path)

# # Remove non-images
# for fname in os.listdir('train/'):
#     if not fname.endswith('.jpg'):
#         os.remove(os.path.join(path, fname))

In [ ]:
# # Resize images - may not need this as dataset_tool can do this for us
# from PIL import Image
# from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True

# def resize():
#     for item in dirs:
#         if os.path.isfile(path + item):
#             im = Image.open(path + item)
#             if im.mode != 'RGB':
#                 im = im.convert('RGB')
#             fname, extension = os.path.splitext(path + item)
#             imResize = im.resize((256, 256), Image.ANTIALIAS)
#             imResize.save(fname + ' resized.jpg', 'JPEG', quality=90)

# resize()

In [ ]:
# # Remove non-resized images
# keyword = 'resized'
# for fname in os.listdir('train/'):
#     if keyword not in fname:
#         os.remove(os.path.join(path, fname))

In [ ]:
# Create a training dataset from folder in Drive and set resolution
!python stylegan3/dataset_tool.py --source drive/MyDrive/train256/ --dest train/ --resolution=256x256

100% 15022/15022 [05:58<00:00, 41.91it/s] 


In [ ]:
# Premade model .pkl file from NVidia
# https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl

In [ ]:
!pip install ninja

     |████████████████████████████████| 108 kB 4.8 MB/s 


In [ ]:
# Initiate training loop. As a test, kimg set to 1 
!python stylegan3/train.py --outdir=training-runs/ --cfg=stylegan3-r --data=train/ --freezed=13 \
      --gpus=1 --batch=32 --gamma=0.5 --mirror=1 --kimg=1 --batch-gpu=16 --snap=10 --metrics=none --cbase=16384\
      --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9988915792636801,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 13
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 0.5,
    "blur_init_s

In [14]:
# Generate an image from seed number, using trained model .pkl file
!python stylegan3/gen_images.py --outdir=out/ --trunc=1 --seeds=2 \
    --network=training-runs/00000-stylegan3-r--gpus1-batch32-gamma0.5/network-snapshot-000001.pkl

Loading networks from "training-runs/00000-stylegan3-r--gpus1-batch32-gamma0.5/network-snapshot-000001.pkl"...
Generating image for seed 2 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
